In [1]:
import pandas as pd
import sqlalchemy
import pymysql

In [2]:
# Define API for SQL database

def create_sql_connection(SQL_USERNAME, SQL_PASSWORD, SQL_DB_NAME, verbose):

    # Connection settings
    SQL_DIALECT = "mysql"  # we are using MySQL
    SQL_DRIVER = "pymysql"  # pymsql provides an interface between MySQL and Python

    # The 
    SQL_URL = "{}+{}://{}:{}@localhost:3306/{}".format(SQL_DIALECT,
                                                       SQL_DRIVER,
                                                       SQL_USERNAME,
                                                       SQL_PASSWORD,
                                                       SQL_DB_NAME)
    engine = sqlalchemy.create_engine(SQL_URL, echo = verbose)
    return engine
    
def run_query(engine, query):
    # runs arbitrary SQL query     
    return engine.execute(query)

def read_sql_query(engine, query):
    # reads data and returns in a pandas dataframe
    return pd.read_sql(sql=query, con=engine)
    
def write_sql_query(engine, df, name):
    # write a pandas dataframe into a SQL server
    df.to_sql(con=engine, name = name, **{"if_exists": "replace", "index": False})

In [3]:
# parameters for connection
SQL_USERNAME = "DTTFUser"
SQL_PASSWORD = "DTTFPassword"
SQL_DB_NAME = "DTFF_DB"
verbose = False
engine = create_sql_connection(SQL_USERNAME, SQL_PASSWORD, SQL_DB_NAME, verbose)

In [4]:

query = 'SELECT * FROM people_info'
query_result = run_query(engine,query)
df = read_sql_query(engine, query)

In [5]:
query_result.fetchall()

[('Jordan Brett Seligmann', datetime.date(1996, 9, 26), 'M', 0),
 ('Silvia Forcina Barrero', datetime.date(1998, 8, 15), 'F', 0),
 ('Filip Sprusansky', datetime.date(1964, 9, 24), 'M', 0),
 ('Villem Adolf Armulik', datetime.date(1967, 6, 25), 'M', 0)]

In [6]:
# add a row to our table
statement2 = 'INSERT INTO people_info (_name, date_of_birth, Sex, Employment_status) VALUES ("Steve Jobs", "2020-12-08", "M", FALSE);'
run_query(engine, statement2)

# show that this updated the table
df1 = read_sql_query(engine, query)
df1


,_name,date_of_birth,Sex,Employment_status
0,Jordan Brett Seligmann,1996-09-26,M,0
1,Silvia Forcina Barrero,1998-08-15,F,0
2,Filip Sprusansky,1964-09-24,M,0
3,Villem Adolf Armulik,1967-06-25,M,0
4,Steve Jobs,2020-12-08,M,0


In [7]:

# delete an entry from our table
run_query(engine, 'DELETE FROM people_info WHERE _name = "Steve Jobs";')
# show that this updated the table
df2 = read_sql_query(engine, query)
df2

,_name,date_of_birth,Sex,Employment_status
0,Jordan Brett Seligmann,1996-09-26,M,0
1,Silvia Forcina Barrero,1998-08-15,F,0
2,Filip Sprusansky,1964-09-24,M,0
3,Villem Adolf Armulik,1967-06-25,M,0


In [8]:
# write data to database using write_sql_query function
write_sql_query(engine, df1, 'people_info_1')

# check database was updated
read_sql_query(engine, 'SELECT * FROM people_info_1')

,_name,date_of_birth,Sex,Employment_status
0,Jordan Brett Seligmann,1996-09-26,M,0
1,Silvia Forcina Barrero,1998-08-15,F,0
2,Filip Sprusansky,1964-09-24,M,0
3,Villem Adolf Armulik,1967-06-25,M,0
4,Steve Jobs,2020-12-08,M,0


In [9]:
# drop newly created table
run_query(engine, 'DROP TABLE IF EXISTS people_info_1;')